In [1]:
## Load libraries
# Interacting with the API
import json
from requests import get

# Working with arrays, tables
import numpy as np
import pandas as pd

# Working with datetime objects
import time
from time import sleep
import pytz
from datetime import datetime, date, timedelta

# Type Hinting for clarity
from typing import Callable

# Binary storage
import pyarrow as pa
import pyarrow.parquet as pq
# Report format
import openpyxl


# FUNCTION DEFINITIONS

def timeshift(report_startdate:int) -> int:
    """
        Shift the timestamp call to api by 15 minutes.
    """
    updatedtime = report_startdate + 900
    return(updatedtime)

def tsconverter(timest : int) -> int:
    """
        Convert the the date from the api to CST timezone to continue to pass it to the api.
    """
    tz = pytz.timezone("America/Chicago")
    dateversion = datetime.fromtimestamp(timest, tz = tz)
    loop_param_timestamp = int(pd.to_datetime(dateversion).timestamp())
    return(loop_param_timestamp)

def initradehist_ts(report_runtime:float, timez:str="America/Chicago") -> int:
    """
        Define a timestamp function that sets the timestamp parameter for Gemini API
        It is hardcoded for the past 24 hours from the run time.
    """
    tz = pytz.timezone(timez)
    start_time = report_runtime
    report_start = datetime.fromtimestamp(start_time, tz = tz)
    hrs24ago = report_start - timedelta(days=1)
    param_timestamp = int(pd.to_datetime(hrs24ago).timestamp())
    return(param_timestamp)
    

def call_tradehistory(symbolurl_struct:dict, key:str, timestamp:int) -> pd.DataFrame:
    """
        Function to call the Gemini Trade History API.
        symbolurl_struct: dictionary containing names of symbols in report.
        key: the symbol to get trade history for.
        timestamp: timestamp from which the trade history API will start to pull data from. 
            It will pull 500 points of data starting from this timestamp.
    """
    response_tradehistory = get(
            url = symbolurl_struct[key],
            params = {
                "since": timestamp,
                "limit_trades":500
                }
            )
    tradehistory_frame = pd.json_normalize(response_tradehistory.json())
    return (tradehistory_frame)

In [2]:

timez = "America/Chicago"
tz = pytz.timezone(timez)

# Set the report start date
report_runtime = datetime.now().replace(tzinfo=tz).timestamp()

# 15 minute intervals
report_interval = 900
# number of 15 minutes in 1 day 
max_calls = 1440/15/2
# Set the APIs we want to connect to:
tradehistory_baseurl = "https://api.gemini.com/v1/"

# Set trade history api endpoint
tradehistory_endpoint = "trades/"

# Set Symbols to be retreived
symbols = ["btcusd","ethbtc"]

# Build url string for each symbol
url_persymbol = ["".join([tradehistory_baseurl,tradehistory_endpoint,sym]) for sym in symbols]

# Create a symbol/url array structure for ongoing programmatic retreival
symbolurl_struct = {}
for symbol,url in zip(symbols,url_persymbol):
    symbolurl_struct[symbol] = url



### MAIN REPORT BUILDER

In [3]:
symbolurl_struct

{'btcusd': 'https://api.gemini.com/v1/trades/btcusd',
 'ethbtc': 'https://api.gemini.com/v1/trades/ethbtc'}

In [4]:
btcusd_response = get(
        url = symbolurl_struct["btcusd"],
        params = {
            "since": initradehist_ts(report_runtime = report_runtime, timez = timez),
            "limit_trades":500
            }
    )

# Turn the raw json response to a pandas dataframe
tradehistory_frame = pd.json_normalize(btcusd_response.json())
tradehistory_frame

report_frame = pd.DataFrame()
temp_apiframe = tradehistory_frame.copy()
api_ts = max(temp_apiframe["timestamp"])
converted_apits = tsconverter(api_ts)
boundar_l = converted_apits - 900*2
boundar_h = converted_apits + 900*2
stop_boundary = report_runtime - 900*2
apicall_count = 1

while boundar_h < stop_boundary or boundar_l < stop_boundary:
        print("\nGetting Trade History\n")
        
        sleep(1)
        shiftedts = timeshift(converted_apits)
        shiftedcheck = max(shiftedts,stop_boundary) == stop_boundary
        if shiftedcheck == False: 
            print(f"\nFinished calling trade history api before next call.\n")
            next_tscall = pd.Timestamp.fromtimestamp(shiftedcheck)
            print(f'Stop calling. Check date: {next_tscall}')
            break
        raw_tradehistory = call_tradehistory(symbolurl_struct,"btcusd", shiftedts)
        api_ts = max(raw_tradehistory["timestamp"])
        newconverted_apits = tsconverter(api_ts)
        print(f"\nMost recent date from trade history: {pd.Timestamp.fromtimestamp(newconverted_apits)}\n")
        boundar_l = newconverted_apits - 900*2
        boundar_h = newconverted_apits + 900*2
        
        if boundar_h > stop_boundary or boundar_l > stop_boundary:
            print(f"Finished calling trade history api.")
            print(pd.Timestamp.fromtimestamp(newconverted_apits))
            break
        
        else:
            report_frame = report_frame.append(raw_tradehistory, ignore_index=True)
            converted_apits = newconverted_apits

        apicall_count = apicall_count + 1
        print(f"\nApi call count: {apicall_count}\n")
        if apicall_count > max_calls:
            break


Getting Trade History


Most recent date from trade history: 2021-05-01 18:09:42


Api call count: 2


Getting Trade History


Most recent date from trade history: 2021-05-01 18:51:48


Api call count: 3


Getting Trade History


Most recent date from trade history: 2021-05-01 19:28:35


Api call count: 4


Getting Trade History


Most recent date from trade history: 2021-05-01 20:19:25


Api call count: 5


Getting Trade History


Most recent date from trade history: 2021-05-01 21:19:32


Api call count: 6


Getting Trade History


Most recent date from trade history: 2021-05-01 22:21:31


Api call count: 7


Getting Trade History


Most recent date from trade history: 2021-05-01 22:44:15


Api call count: 8


Getting Trade History


Most recent date from trade history: 2021-05-01 23:09:42


Api call count: 9


Getting Trade History


Most recent date from trade history: 2021-05-01 23:57:32


Api call count: 10


Getting Trade History


Most recent date from trade history: 2021-05-02

In [6]:
date_check = datetime.fromtimestamp(report_frame['timestamp'].iloc[-1])
print(date_check)
csv_frame = report_frame.copy()

2021-05-02 14:49:46


In [7]:
def ts_convert(timest):
    dateversion = datetime.fromtimestamp(timest)
    return(dateversion)

def tsms_convert(time_ms):
    s = time_ms / 1000.0
    dttime = datetime.fromtimestamp(s)
    return dttime.strftime('%Y-%m-%d %H:%M:%S.%f')


print(csv_frame.head())
print("\n")

    timestamp    timestampms          tid     price      amount exchange type
0  1619910582  1619910582742  37081221885  57692.57  0.00237879   gemini  buy
1  1619910582  1619910582742  37081221883  57692.57  0.00002402   gemini  buy
2  1619910559  1619910559452  37081192530  57701.65  0.00139175   gemini  buy
3  1619910553  1619910553783  37081187629  57707.31  0.00759975   gemini  buy
4  1619910549  1619910549784  37081185344  57710.20  0.00167262   gemini  buy




### Converting the dates to a readable format for CSV 

In [8]:
readable_timestamp = pd.Series(map(ts_convert,csv_frame['timestamp']))

In [9]:
# CHECK
print(readable_timestamp[:-4])

0       2021-05-01 18:09:42
1       2021-05-01 18:09:42
2       2021-05-01 18:09:19
3       2021-05-01 18:09:13
4       2021-05-01 18:09:09
                ...        
12491   2021-05-02 14:50:26
12492   2021-05-02 14:50:26
12493   2021-05-02 14:50:25
12494   2021-05-02 14:50:25
12495   2021-05-02 14:50:21
Length: 12496, dtype: datetime64[ns]


In [10]:
# Assign
csv_frame['timestamp'] = readable_timestamp

In [11]:
# Repeat for timestampms
readable_timestampms = pd.Series(map(tsms_convert, csv_frame['timestampms']))

In [12]:
print(readable_timestampms[:-4])

0        2021-05-01 18:09:42.742000
1        2021-05-01 18:09:42.742000
2        2021-05-01 18:09:19.452000
3        2021-05-01 18:09:13.783000
4        2021-05-01 18:09:09.784000
                    ...            
12491    2021-05-02 14:50:26.462000
12492    2021-05-02 14:50:26.462000
12493    2021-05-02 14:50:25.731000
12494    2021-05-02 14:50:25.136000
12495    2021-05-02 14:50:21.624000
Length: 12496, dtype: object


In [13]:
csv_frame['timestampms'] = readable_timestampms

In [14]:
csv_frame.head()

,timestamp,timestampms,tid,price,amount,exchange,type
0,2021-05-01 18:09:42,2021-05-01 18:09:42.742000,37081221885,57692.57,0.00237879,gemini,buy
1,2021-05-01 18:09:42,2021-05-01 18:09:42.742000,37081221883,57692.57,0.00002402,gemini,buy
2,2021-05-01 18:09:19,2021-05-01 18:09:19.452000,37081192530,57701.65,0.00139175,gemini,buy
3,2021-05-01 18:09:13,2021-05-01 18:09:13.783000,37081187629,57707.31,0.00759975,gemini,buy
4,2021-05-01 18:09:09,2021-05-01 18:09:09.784000,37081185344,57710.20,0.00167262,gemini,buy


In [15]:
# If it looks okay, export to CSV
csv_frame.to_csv('tradehistory_daily_btcusd_gemini_210502.csv', index=False)

### Save in a persistant binary format

In [16]:
pqtable = pa.Table.from_pandas(report_frame)
pq.write_table(pqtable, 'tradehistory_daily_btcusd_gemini_210502.parquet')

# Discussion

- Running again for ethbtc
